In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import pandas as pd
#import serial
import time

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp4/weights/last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /Users/mikerichardson/.cache/torch/hub/master.zip
YOLOv5 🚀 2021-12-13 torch 1.9.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7088338 parameters, 0 gradients, 16.0 GFLOPs
Adding AutoShape... 


In [3]:
img = os.path.join('data', 'images', 'image9a5e4da2-577c-11ec-aecb-1e00a2015b79.jpg')

/Users/mikerichardson/opt/anaconda3/bin/python: Error while finding module specification for 'serial.tools.list_ports' (ModuleNotFoundError: No module named 'serial')


In [4]:
results = model(img)

In [5]:
object_array = results.pandas().xyxy[0] #extracts object data in np 

In [ ]:
rounded_array = np.around(object_array, -2) #rounds the array to simplify

In [6]:
print(object_array)

         xmin        ymin        xmax        ymax  confidence  class  \
0  167.617157  126.168694  241.149261  208.923492    0.971933     16   
1  487.354309   41.608566  543.469055   99.118774    0.970163     18   
2  488.588165  349.535950  541.926270  396.426575    0.962295     18   
3  306.141144  178.444748  350.039948  222.967545    0.959421     15   
4  227.054932  235.575424  297.402588  282.051941    0.942360     16   
5  436.960358  409.353302  468.622223  440.990570    0.940581     15   
6  408.342499  394.987518  437.747650  420.356110    0.924324     16   
7  557.803467  350.557526  599.980469  388.319733    0.917312     20   
8  162.436203  209.745438  208.682175  258.597992    0.636365     18   
9  162.368790  210.140533  209.447037  258.773651    0.475255     16   

           name  
0         spots  
1  orange_green  
2  orange_green  
3           red  
4         spots  
5           red  
6         spots  
7        yellow  
8  orange_green  
9         spots  


In [4]:
col_1 = [0x03, 0x07, 0x0b, 0x0f, 0x00]
col_2 = [0x04, 0x08, 0x0c, 0x10, 0x00]
col_3 = [0x05, 0x09, 0x0d, 0x11, 0x00]
col_4 = [0x06, 0x0a, 0x0e, 0x12, 0x00]
stop = [0x00, 0x00]

In [ ]:
test_row = rounded_array.iloc[0]

In [2]:
ser = serial.Serial(port='/dev/cu.usbmodem11401',baudrate=9600, timeout=5)

SerialException: [Errno 2] could not open port /dev/cu.usbmodem11401: [Errno 2] No such file or directory: '/dev/cu.usbmodem11401'

In [6]:
def update_display(row):
    
    if row.xmin == 100 and row.xmax == 100:
        ser.write(serial.to_bytes(col_1))
    elif row.xmin == 100 and row.xmax == 200:
        ser.write(serial.to_bytes(col_1))
        ser.write(serial.to_bytes(col_2))
    elif row.xmin == 100 and row.xmax == 300:
        ser.write(serial.to_bytes(col_1))
        ser.write(serial.to_bytes(col_2))
        ser.write(serial.to_bytes(col_3))
    elif row.xmin == 100 and row.xmax == 400:
        ser.write(serial.to_bytes(col_1))
        ser.write(serial.to_bytes(col_2))
        ser.write(serial.to_bytes(col_3))
        ser.write(serial.to_bytes(col_4))
        
    elif row.xmin == 200 and row.xmax == 200:
        ser.write(serial.to_bytes(col_2))
    elif row.xmin == 200 and row.xmax == 300:
        ser.write(serial.to_bytes(col_2))
        ser.write(serial.to_bytes(col_3))
    elif row.xmin == 200 and row.xmax == 400:
        ser.write(serial.to_bytes(col_2))
        ser.write(serial.to_bytes(col_3))
        ser.write(serial.to_bytes(col_4))
        
    elif row.xmin == 300 and row.xmax == 300:
        ser.write(serial.to_bytes(col_3))
    elif row.xmin == 300 and row.xmax == 400:
        ser.write(serial.to_bytes(col_3))
        ser.write(serial.to_bytes(col_4))

    elif row.xmin == 400 and row.xmax == 400:
        ser.write(serial.to_bytes(col_4))
        
    else:
        ser.write(serial.to_bytes(stop))



In [ ]:
update_display(test_row)

In [ ]:
for index, row in rounded_array.iterrows():
    update_display(row)
    time.sleep(1)

In [ ]:
ser.write(serial.to_bytes(stop))

In [ ]:
results

In [ ]:
%matplotlib inline
plt.imshow(np.squeeze(results.render())[300:400, 120:])
plt.show()


In [4]:
cap = cv2.VideoCapture('Users\install\Desktop\Climbing Study\data\hold_footage_2.mov')

while cap.isOpened():
    ret, frame = cap.read()
    
    #make detections
    results = model(frame) #[300:400, 120:520] add after frame to crop
    
    cv2.imshow('YOLO', np.squeeze(results.render())) #change string to rename the window
 #   object_array = results.pandas().xyxy[0]
 #   rounded_array = np.around(object_array, -2)
    
 #   for index, row in rounded_array.iterrows():
  #      update_display(row)
        
    
    
                                        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
for i in range(1,5):
    cv2.waitKey(1)
#ser.write(serial.to_bytes(stop))

In [ ]:
cv2.destroyAllWindows()